In [9]:
# Importar bibliotecas
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [10]:
# Instalar Webdriver
service = Service(ChromeDriverManager().install())

In [11]:
# Abrir aba Web
driver = webdriver.Chrome(service=service)
# Localizar website e entrar
website = 'https://www.interpol.int/How-we-work/Notices/Yellow-Notices/View-Yellow-Notices'
driver.get(website)
# Maximizar aba Web
driver.maximize_window()
# Inplicit wait de 5 segundos para renderizar a página ao abrir
time.sleep(5)
# Localizar botão e fechar aba de Cookies da página
search_button = driver.find_element(by='xpath', value="//i[@class='privacy-cookie-banner__icon-close']")
search_button.click()

# Criar listas vazias para armazenar a informação que queremos
name = []
ages = []
country = []
country_2 = []

In [12]:
# Encontrar todos os paises no seletor de nacionalidade da página e anexá los na lista country_2
select_countries = Select(driver.find_element(by='xpath', value='//select[@id="nationalities"]'))

for option in select_countries.options:
    country_2.append(option.text)

# Remover o primeiro elemento da lista country_2 (é um elemento vazio)
country_2.pop(0)

''

In [13]:
# Itera pelos países na lista country_2
for pais in country_2:
    select_countries.select_by_visible_text(pais)

    # Loop entre idades de 0 a 110 anos
    for age in range(0, 111):
        min_age_input = driver.find_element(by='xpath', value='//input[@id="ageMin"]')
        min_age_input.clear()  # Limpa o campo de idade mínima
        min_age_input.send_keys(str(age))  # Insere a idade mínima

    # Defina max_age como age + 1
        max_age = age + 1

        max_age_input = driver.find_element(by='xpath', value='//input[@id="ageMax"]')
        max_age_input.clear()  # Limpa o campo de idade máxima
        max_age_input.send_keys(str(max_age))  # Insere a idade máxima

        # Localiza o botão de pesquisa e clika nele
        search_button = driver.find_element(by='xpath', value="//button[@id='submit' and @type='submit']")
        search_button.click()

        time.sleep(5)

        # Localiza o elemento da barra de paginação usando XPath
        pagination = driver.find_element(by='xpath', value='//div[contains(@id, "paginationPanel")]')

        # Localiza todos os elementos <li> dentro do elemento de paginação
        pages = pagination.find_elements(by='xpath', value='.//li')

        # Obtenha o número da primeira e última página
        initial_page = 1

        # Verifique se há pelo menos dois elementos em 'pages' antes de acessar 'pages[-2]'
        if len(pages) >= 2:
            last_page = int(pages[-2].text)
        else:
            last_page = initial_page


        # Loop While para paginar todas as paginas do Website
        while initial_page <= last_page:

            # Try e Except para pular erros quando a página se encontra vazia (sem a presença de elementos para extrair)
            try:
                # Explicit Wait para esperar os elementos desejados aparecerem na página
                container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@class="redNoticesList__ajaxContainer"]')))
                products = WebDriverWait(container, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="redNoticeItem"]')))
            except:
                pass

            # Try e Except para pular erros quando a página se encontra vazia (sem a presença de elementos para extrair)
            try:
                # Encontrar os elementos que desejamos exrair por xpath e anexar todos em suas devidas listas (criadas acima)
                for product in products:
                    name.append(product.find_element(by='xpath', value=".//a[@class='redNoticeItem__labelLink']").text)
                    ages.append(product.find_element(by='xpath', value='.//span[@class="age"]').text)
                    country.append(product.find_element(by='xpath', value='.//span[@class="nationalities"]').text)
            except:
                pass

                # Try e Except para evitar erros quando o prgrama chega na ultima aba e não existe mais o botão para uma próxima pagina
            try:
                # Encontra o botão que vai para a próxima aba (direita) e clika nele
                next_button = driver.find_element(by='xpath', value="//a[@class='nextIndex right-arrow']")
                next_button.click()
            except:
                pass


            # Parte final da paginação, diz ao loop while para somar +1 na contagem de páginas e ir para a proxima repetição
            initial_page = initial_page + 1
driver.quit()

In [ ]:
# Transforma as listas já preenchidas com informação em um documento CSV, utilizando o Pandas.
max_length = max(len(name), len(ages), len(country))

while len(name) < max_length:
    name.append('')
while len(ages) < max_length:
    ages.append('')
while len(country) < max_length:
    country.append('')

df_list = pd.DataFrame({'Name': name, 'Age': ages, 'Country': country})
df_list.to_csv('Interpol_yellow_Notices.csv', index=False)